In [20]:
from tqdm import tqdm_notebook as tqdm
from IPython.display import Markdown, display
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras

In [21]:
def printmd(string):
    display(Markdown(string))

При чтении будем хранить данные в переменной `CACHE`, чтобы ускорить загрузку. В случае их обновления, достаточно вызвать `read(override=True)`

In [22]:
SMALL_DATASET_DIR = '../../nmnist/notMNIST_small/'
LARGE_DATASET_DIR = '../../nmnist/notMNIST_large/'
CACHE = {}
LABEL_MAP = {}
INV_LABEL_MAP = {}

In [33]:
def read(data_dir, override=False):
    f_v = 0
    global CACHE
    if not CACHE.get(data_dir, []) or override:
        CACHE[data_dir] = []
        X, y = [], []
        for f in tqdm(os.listdir(data_dir), desc='Letter'):
            if not f.startswith('.'):
                img_dir = os.path.join(data_dir, f)
                for img in os.listdir(img_dir):
                    img_path = os.path.join(img_dir, img)
                    data = cv2.imread(img_path, 0)
                    if data is None:
                        continue
                    X.append(data * 2 / 255 - 1)
                    if LABEL_MAP.get(f) is None:
                        LABEL_MAP[f] = f_v
                        INV_LABEL_MAP[f_v] = f
                        f_v += 1
                    y.append(LABEL_MAP[f])
        CACHE[data_dir].append(np.array(X))
        CACHE[data_dir].append(np.array(y))
    return CACHE[data_dir][0], CACHE[data_dir][1]

In [34]:
def get_data(data_dir, verbose=False, override=False):
    X, y = read(data_dir, override=override)
    assert X.shape[0] == y.shape[0]
    N = X.shape[0]
    if verbose:
        print(X.shape)
        print(y.shape)
        print(X[:5])
        print(y[:5])
        print(np.unique(y))
    return X, y

In [35]:
def get_split_data(data_dir, size=(0.7, 0.3), verbose=False, random_state=6, override=False):
    X, y = shuffle(*get_data(data_dir, verbose=verbose, override=override), random_state=random_state)
    assert abs(np.sum(size) - 1.0) < 0.001
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size[1], random_state=random_state)
    return X_train, y_train, X_test, y_test

In [36]:
X_train, y_train, X_test, y_test = get_split_data(SMALL_DATASET_DIR, override=True)

Построим 5 слоев с функциями активации `ReLU` и `softmax` на последнем слое, коэффициент обучения $0.001$, $100$ эпох:

In [47]:
lr = 0.001
epochs = 100
printmd('**Building with new architecture...**')
print('512(relu) - 256(relu) - 64(relu) - 32(relu) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
printmd(f'**Loss**: {loss}')
printmd(f'**Accuracy**: {acc}')
print('=======================')

**Building with new architecture...**

512(relu) - 256(relu) - 64(relu) - 32(relu) - 10(softmax)
Learning rate: 0.001
Epochs: 100

Epoch 1/100
13106/13106 [==============================] - 4s 295us/step - loss: 2.0085 - acc: 0.3497
Epoch 2/100
13106/13106 [==============================] - 3s 202us/step - loss: 1.4721 - acc: 0.6099
Epoch 3/100
13106/13106 [==============================] - 3s 199us/step - loss: 1.0691 - acc: 0.7716
Epoch 4/100
13106/13106 [==============================] - 2s 190us/step - loss: 0.8325 - acc: 0.8294
Epoch 5/100
13106/13106 [==============================] - 3s 212us/step - loss: 0.7102 - acc: 0.8438
Epoch 6/100
13106/13106 [==============================] - 3s 209us/step - loss: 0.6426 - acc: 0.8530
Epoch 7/100
13106/13106 [==============================] - 3s 221us/step - loss: 0.5988 - acc: 0.8564
Epoch 8/100
13106/13106 [==============================] - 3s 235us/step - loss: 0.5680 - acc: 0.8625
Epoch 9/100
13106/13106 [==============================] - 3s 215us/step - loss: 0.5451 - ac

13106/13106 [==============================] - 3s 216us/step - loss: 0.2432 - acc: 0.9326
Epoch 80/100
13106/13106 [==============================] - 3s 198us/step - loss: 0.2411 - acc: 0.9313
Epoch 81/100
13106/13106 [==============================] - 3s 208us/step - loss: 0.2394 - acc: 0.9321
Epoch 82/100
13106/13106 [==============================] - 3s 209us/step - loss: 0.2377 - acc: 0.9328
Epoch 83/100
13106/13106 [==============================] - 3s 205us/step - loss: 0.2352 - acc: 0.9341
Epoch 84/100
13106/13106 [==============================] - 3s 212us/step - loss: 0.2329 - acc: 0.9342
Epoch 85/100
13106/13106 [==============================] - 3s 205us/step - loss: 0.2312 - acc: 0.9348
Epoch 86/100
13106/13106 [==============================] - 3s 207us/step - loss: 0.2293 - acc: 0.9347
Epoch 87/100
13106/13106 [==============================] - 3s 204us/step - loss: 0.2268 - acc: 0.9363
Epoch 88/100
13106/13106 [==============================] - 3s 207us/step - loss: 0.22

**Loss**: 0.3476183395700157

**Accuracy**: 0.9013883944464223

Изменим коэффициент обучения на больший `lr = 0.1`:

In [48]:
lr = 0.1
epochs = 100
printmd('**Building with new architecture...**')
print('512(relu) - 256(relu) - 64(relu) - 32(relu) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
printmd(f'**Loss**: {loss}')
printmd(f'**Accuracy**: {acc}')
print('=======================')

**Building with new architecture...**

512(relu) - 256(relu) - 64(relu) - 32(relu) - 10(softmax)
Learning rate: 0.1
Epochs: 100

Epoch 1/100
13106/13106 [==============================] - 5s 366us/step - loss: 0.7723 - acc: 0.7586
Epoch 2/100
13106/13106 [==============================] - 3s 224us/step - loss: 0.4170 - acc: 0.8749
Epoch 3/100
13106/13106 [==============================] - 3s 210us/step - loss: 0.3510 - acc: 0.8910
Epoch 4/100
13106/13106 [==============================] - 3s 211us/step - loss: 0.3143 - acc: 0.8997
Epoch 5/100
13106/13106 [==============================] - 3s 222us/step - loss: 0.2814 - acc: 0.9117
Epoch 6/100
13106/13106 [==============================] - 3s 202us/step - loss: 0.2575 - acc: 0.9176
Epoch 7/100
13106/13106 [==============================] - 3s 225us/step - loss: 0.2320 - acc: 0.9265
Epoch 8/100
13106/13106 [==============================] - 3s 249us/step - loss: 0.2189 - acc: 0.9283
Epoch 9/100
13106/13106 [==============================] - 3s 241us/step - loss: 0.2087 - acc:

13106/13106 [==============================] - 3s 192us/step - loss: 0.0103 - acc: 0.9974
Epoch 80/100
13106/13106 [==============================] - 3s 193us/step - loss: 0.0052 - acc: 0.9986
Epoch 81/100
13106/13106 [==============================] - 3s 198us/step - loss: 0.0062 - acc: 0.9979
Epoch 82/100
13106/13106 [==============================] - 3s 193us/step - loss: 0.0080 - acc: 0.9978
Epoch 83/100
13106/13106 [==============================] - 3s 193us/step - loss: 0.0059 - acc: 0.9982
Epoch 84/100
13106/13106 [==============================] - 3s 194us/step - loss: 0.0025 - acc: 0.9992
Epoch 85/100
13106/13106 [==============================] - 3s 198us/step - loss: 0.0021 - acc: 0.9993
Epoch 86/100
13106/13106 [==============================] - 3s 192us/step - loss: 0.0019 - acc: 0.9995
Epoch 87/100
13106/13106 [==============================] - 3s 197us/step - loss: 0.0014 - acc: 0.9995
Epoch 88/100
13106/13106 [==============================] - 3s 194us/step - loss: 0.00

**Loss**: 0.5417178698837813

**Accuracy**: 0.9229263082947669

Точность увеличилась, попробуем найти оптимальный коэффициент обучения:

In [54]:
lr = 0.2
epochs = 100
printmd('**Building with new architecture...**')
print('512(relu) - 256(relu) - 64(relu) - 32(relu) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
printmd(f'**Loss**: {loss}')
printmd(f'**Accuracy**: {acc}')
print('=======================')

**Building with new architecture...**

512(relu) - 256(relu) - 64(relu) - 32(relu) - 10(softmax)
Learning rate: 0.2
Epochs: 100

Epoch 1/100
13106/13106 [==============================] - 4s 321us/step - loss: 1.2485 - acc: 0.6035
Epoch 2/100
13106/13106 [==============================] - 2s 189us/step - loss: 0.4966 - acc: 0.8528
Epoch 3/100
13106/13106 [==============================] - 3s 195us/step - loss: 0.4104 - acc: 0.8766
Epoch 4/100
13106/13106 [==============================] - 3s 193us/step - loss: 0.3694 - acc: 0.8875
Epoch 5/100
13106/13106 [==============================] - 3s 221us/step - loss: 0.3338 - acc: 0.8970
Epoch 6/100
13106/13106 [==============================] - 3s 227us/step - loss: 0.3085 - acc: 0.9042
Epoch 7/100
13106/13106 [==============================] - 3s 221us/step - loss: 0.2867 - acc: 0.9109
Epoch 8/100
13106/13106 [==============================] - 3s 203us/step - loss: 0.2640 - acc: 0.9152
Epoch 9/100
13106/13106 [==============================] - 3s 204us/step - loss: 0.2793 - acc:

13106/13106 [==============================] - 3s 205us/step - loss: 0.0542 - acc: 0.9857
Epoch 80/100
13106/13106 [==============================] - 3s 201us/step - loss: 0.0348 - acc: 0.9899
Epoch 81/100
13106/13106 [==============================] - 3s 196us/step - loss: 0.0236 - acc: 0.9933
Epoch 82/100
13106/13106 [==============================] - 3s 201us/step - loss: 0.0234 - acc: 0.9928
Epoch 83/100
13106/13106 [==============================] - 3s 219us/step - loss: 0.0295 - acc: 0.9914
Epoch 84/100
13106/13106 [==============================] - 3s 214us/step - loss: 0.0628 - acc: 0.9854
Epoch 85/100
13106/13106 [==============================] - 3s 198us/step - loss: 0.0466 - acc: 0.9871
Epoch 86/100
13106/13106 [==============================] - 3s 199us/step - loss: 0.0467 - acc: 0.9875
Epoch 87/100
13106/13106 [==============================] - 3s 196us/step - loss: 0.0324 - acc: 0.9916
Epoch 88/100
13106/13106 [==============================] - 3s 199us/step - loss: 0.01

**Loss**: 0.5655547246731695

**Accuracy**: 0.9213243147027412

Как видно, точность упала. Поэтому экспериментально будем считать, что коэффициент обучения, увеличение которого ухудшает точность, равен `lr_opt = 0.1`. 

Далее попробуем применить другие функции активации, например `tanh`:

In [55]:
lr = 0.1
epochs = 100
printmd('**Building with new architecture...**')
print('512(tanh) - 128(tanh) - 32(tanh) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.tanh),
    keras.layers.Dense(128, activation=tf.nn.tanh),
    keras.layers.Dense(32, activation=tf.nn.tanh),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
printmd(f'**Loss**: {loss}')
printmd(f'**Accuracy**: {acc}')
print('=======================')

**Building with new architecture...**

512(tanh) - 128(tanh) - 32(tanh) - 10(softmax)
Learning rate: 0.1
Epochs: 100

Epoch 1/100
13106/13106 [==============================] - 4s 329us/step - loss: 0.6883 - acc: 0.8031
Epoch 2/100
13106/13106 [==============================] - 3s 192us/step - loss: 0.4123 - acc: 0.8810
Epoch 3/100
13106/13106 [==============================] - 3s 204us/step - loss: 0.3490 - acc: 0.8944
Epoch 4/100
13106/13106 [==============================] - 4s 341us/step - loss: 0.3126 - acc: 0.9062 1s
Epoch 5/100
13106/13106 [==============================] - 3s 230us/step - loss: 0.2793 - acc: 0.9161
Epoch 6/100
13106/13106 [==============================] - 3s 222us/step - loss: 0.2563 - acc: 0.9212
Epoch 7/100
13106/13106 [==============================] - 3s 220us/step - loss: 0.2345 - acc: 0.9265
Epoch 8/100
13106/13106 [==============================] - 3s 214us/step - loss: 0.2161 - acc: 0.9337
Epoch 9/100
13106/13106 [==============================] - 4s 305us/step - loss: 0.1937 - acc: 0.9409


13106/13106 [==============================] - 3s 231us/step - loss: 0.0226 - acc: 0.9938
Epoch 80/100
13106/13106 [==============================] - 3s 203us/step - loss: 0.0274 - acc: 0.9918
Epoch 81/100
13106/13106 [==============================] - 2s 181us/step - loss: 0.0039 - acc: 0.9992
Epoch 82/100
13106/13106 [==============================] - 2s 184us/step - loss: 0.0023 - acc: 0.9995
Epoch 83/100
13106/13106 [==============================] - 3s 235us/step - loss: 0.0021 - acc: 0.9995
Epoch 84/100
13106/13106 [==============================] - 3s 248us/step - loss: 0.0026 - acc: 0.9992
Epoch 85/100
13106/13106 [==============================] - 2s 178us/step - loss: 0.0019 - acc: 0.9996
Epoch 86/100
13106/13106 [==============================] - 2s 176us/step - loss: 0.0017 - acc: 0.9995
Epoch 87/100
13106/13106 [==============================] - 2s 177us/step - loss: 0.0020 - acc: 0.9994
Epoch 88/100
13106/13106 [==============================] - 2s 176us/step - loss: 0.00

**Loss**: 0.4921583931814002

**Accuracy**: 0.9195443218227127

В целом результаты не улучшились

Попробуем добавить dropout (используем небольшой коэффициент, как рекомендуется здесь: https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/

```Generally, use a small dropout value of 20%-50% of neurons with 20% providing a good starting point. A probability too low has minimal effect and a value too high results in under-learning by the network.```)

In [57]:
lr = 0.1
epochs = 200
printmd('**Building with new architecture...**')
print('512(relu) - 128(relu) - 32(tanh) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation=tf.nn.tanh),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
printmd(f'**Loss**: {loss}')
printmd(f'**Accuracy**: {acc}')
print('=======================')

**Building with new architecture...**

512(relu) - 128(relu) - 32(tanh) - 10(softmax)
Learning rate: 0.1
Epochs: 100

Epoch 1/100
13106/13106 [==============================] - 5s 387us/step - loss: 0.8286 - acc: 0.7500
Epoch 2/100
13106/13106 [==============================] - 5s 381us/step - loss: 0.4943 - acc: 0.8556
Epoch 3/100
13106/13106 [==============================] - 4s 290us/step - loss: 0.4319 - acc: 0.8705
Epoch 4/100
13106/13106 [==============================] - 4s 286us/step - loss: 0.3848 - acc: 0.8855
Epoch 5/100
13106/13106 [==============================] - 4s 276us/step - loss: 0.3761 - acc: 0.8862
Epoch 6/100
13106/13106 [==============================] - 4s 271us/step - loss: 0.3491 - acc: 0.8958
Epoch 7/100
13106/13106 [==============================] - 4s 280us/step - loss: 0.3239 - acc: 0.9006
Epoch 8/100
13106/13106 [==============================] - 3s 261us/step - loss: 0.3128 - acc: 0.9059
Epoch 9/100
13106/13106 [==============================] - 3s 251us/step - loss: 0.3061 - acc: 0.9067
Epo

13106/13106 [==============================] - 3s 254us/step - loss: 0.0794 - acc: 0.9733
Epoch 80/100
13106/13106 [==============================] - 3s 256us/step - loss: 0.0955 - acc: 0.9686
Epoch 81/100
13106/13106 [==============================] - 3s 258us/step - loss: 0.0833 - acc: 0.9713
Epoch 82/100
13106/13106 [==============================] - 3s 253us/step - loss: 0.0811 - acc: 0.9731
Epoch 83/100
13106/13106 [==============================] - 3s 262us/step - loss: 0.0843 - acc: 0.9725
Epoch 84/100
13106/13106 [==============================] - 3s 254us/step - loss: 0.0862 - acc: 0.9715
Epoch 85/100
13106/13106 [==============================] - 3s 255us/step - loss: 0.0824 - acc: 0.9715
Epoch 86/100
13106/13106 [==============================] - 3s 266us/step - loss: 0.0766 - acc: 0.9750
Epoch 87/100
13106/13106 [==============================] - 3s 264us/step - loss: 0.0761 - acc: 0.9744
Epoch 88/100
13106/13106 [==============================] - 3s 256us/step - loss: 0.07

**Loss**: 0.4299501164658274

**Accuracy**: 0.9120683517265931

Попробуем применить динамический (адаптивный) коэффициент обучения. Для этого воспользуемся методом `Adam`:

In [64]:
lr = 0.001
epochs = 100
printmd('**Building with new architecture...**')
print('512(relu) - 128(relu) - 32(tanh) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.tanh),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
loss_method = 'sparse_categorical_crossentropy'
model.compile(optimizer=tf.train.AdamOptimizer(lr), 
              loss=loss_method,
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
printmd(f'**Loss**: {loss}')
printmd(f'**Accuracy**: {acc}')
print('=======================')

**Building with new architecture...**

512(relu) - 128(relu) - 32(tanh) - 10(softmax)
Learning rate: 0.001
Epochs: 100

Epoch 1/100
13106/13106 [==============================] - 9s 674us/step - loss: 0.8378 - acc: 0.7442
Epoch 2/100
13106/13106 [==============================] - 6s 468us/step - loss: 0.5344 - acc: 0.8432
Epoch 3/100
13106/13106 [==============================] - 7s 500us/step - loss: 0.4681 - acc: 0.8613
Epoch 4/100
13106/13106 [==============================] - 8s 576us/step - loss: 0.4523 - acc: 0.8653
Epoch 5/100
13106/13106 [==============================] - 8s 596us/step - loss: 0.4151 - acc: 0.8756
Epoch 6/100
13106/13106 [==============================] - 7s 520us/step - loss: 0.3984 - acc: 0.8814
Epoch 7/100
13106/13106 [==============================] - 6s 487us/step - loss: 0.3894 - acc: 0.8825
Epoch 8/100
13106/13106 [==============================] - 6s 486us/step - loss: 0.3865 - acc: 0.8830
Epoch 9/100
13106/13106 [==============================] - 8s 581us/step - loss: 0.3674 - acc: 0.8875
E

13106/13106 [==============================] - 7s 550us/step - loss: 0.2142 - acc: 0.9301
Epoch 80/100
13106/13106 [==============================] - 7s 528us/step - loss: 0.2044 - acc: 0.9332
Epoch 81/100
13106/13106 [==============================] - 11s 838us/step - loss: 0.2163 - acc: 0.9303
Epoch 82/100
13106/13106 [==============================] - 8s 595us/step - loss: 0.2047 - acc: 0.9337
Epoch 83/100
13106/13106 [==============================] - 9s 649us/step - loss: 0.2066 - acc: 0.9316
Epoch 84/100
13106/13106 [==============================] - 6s 483us/step - loss: 0.2150 - acc: 0.9303
Epoch 85/100
13106/13106 [==============================] - 7s 520us/step - loss: 0.2004 - acc: 0.9341
Epoch 86/100
13106/13106 [==============================] - 8s 590us/step - loss: 0.2025 - acc: 0.9351
Epoch 87/100
13106/13106 [==============================] - 6s 451us/step - loss: 0.1866 - acc: 0.9413
Epoch 88/100
13106/13106 [==============================] - 7s 546us/step - loss: 0.1

**Loss**: 0.33870873077457253

**Accuracy**: 0.9133143466577364

<div style="text-align: right"> @chiselko6 </div>